In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime

In [2]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

### Visit the NASA Mars News Site

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [5]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [6]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8785/nasas-perseverance-rover-is-midway-to-mars/" target="_self">NASA's Perseverance Rover Is Midway to Mars </a></div>

In [7]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Is Midway to Mars "

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination."

### JPL Space Images Featured Image

In [9]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [11]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [12]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [13]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA09178_hires.jpg'

In [14]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA09178_hires.jpg'

### Mars Facts

In [15]:
df = pd.read_html('http://space-facts.com/mars/')[0]

df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [16]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

### Mars Weather

In [12]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

In [13]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [15]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('pre', class_='embed_code').text
weather_table

"<iframe src='https://mars.nasa.gov/layout/embed/image/insightweather/' width='800' height='530'  scrolling='no' frameborder='0'></iframe>"

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [159]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [160]:
begin_time = datetime.datetime.now()
# 2 make a list to hold the urls
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
# parse the page html
html = browser.html
hemi_soup = soup(html, 'html.parser')
# find the container that holds all the links
hemi_parent = hemi_soup.find('div', class_='collapsible results')
# find the container that holds each individual link
hemi_child = hemi_parent.find_all('a')
#loop through all the results and do a bunch of stuff:
for item in hemi_child:
    #grab the relative link to the fullsize image
    link = item.get('href')
    # make the full url 
    item_url = f'https://astrogeology.usgs.gov{link}'
    #send the browser to that link
    browser.visit(item_url)
    #parse the page html
    html = browser.html
    hemi_soup = soup(html, 'html.parser')
    # get the relative image url
    img_url_rel = hemi_soup.select_one('img.wide-image').get("src")
    # get the title
    img_title = hemi_soup.select_one('h2.title').text
    # Use the base URL to create an absolute URL
    img_url = f'https://astrogeology.usgs.gov{img_url_rel}'
    # append the url list with a dictionary containing the title and image url
    hemisphere_image_urls.append({"img_url" :img_url, "title" : img_title})
# each child container had 2 identicle links, drop the dupes
hemisphere_image_urls = [i for n, i in enumerate(hemisphere_image_urls) if i not in hemisphere_image_urls[n + 1:]] 
#print elapsed time 
print(datetime.datetime.now() - begin_time)

0:00:06.048637


In [161]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [24]:
# 5. Quit the browser
browser.quit()